In [ ]:
from torchvision.ops import batched_nms
%load_ext autoreload
%autoreload 2

import sys
import os
import matplotlib.patches as patches
sys.path.append(".")

from Models.splinedist import *
import pytorch_lightning as pl
import torch
from Datasets.CISD import *
from Datasets.DSB18 import * 
from torchvision import io
import glob

In [ ]:
# # pl.seed_everything(47)
# data = CISD_datamodule(path="Datasets/CISD/CISD.json", root="Datasets/CISD/center_slice", contourSize=500)
# batch_x, batch_y = next(iter(data.val_dataloader()))
# showBatch((batch_x, batch_y))
# !pip install shapely

# Evaluation on the Dataset

In [ ]:

modelName = "Experiments/Models_final21/weights/splineDist-CISD10-epoch=101-loss_val=5.61.ckpt"
# modelName = "Experiments/Models_final2/weights/splineDist-CISD4-epoch=86-loss_val=0.08.ckpt"
model = SplineDist.load_from_checkpoint(
    checkpoint_path=modelName)
device = "cuda:4"

# Evaluation on costum images

In [ ]:
transforms = get_transforms(0.5, 0.5)

# model = model.to(device)
model = model.to(device)
MAX_CONTOUR_SIZE = 256
threshold = 0.2
nms_threshold = 0.4
num_samples = 200
# MAX_CONTOUR_SIZE = 500
for image in glob.glob("inferenceCISD/*.jpg"):
    print(image)
    img = io.read_image(image, io.ImageReadMode.RGB).float()/255
    img = transforms(img).unsqueeze(0)
    img = img.to(device)
    model.eval()

    with torch.no_grad():
        (objectProbas, angles, distances, controlPoints, objectOverlap) = model(img)
        objectProbas = torch.sigmoid(objectProbas)
        objectOverlap = torch.sigmoid(objectOverlap)
        contours = getContourSamples(controlPoints, model.B3M).permute(1, 2, 3, 0, 4)
        contours = contours.reshape(-1, MAX_CONTOUR_SIZE, 2)
        
                
        overlap = objectOverlap[0, 0].detach()
        overlap = (overlap - overlap.min())/(overlap.max() - overlap.min())
        objprob = objectProbas[0, 0].detach()
        objprop = objprob * (1 - overlap)
        
        objprop_t = objprop * (objprop > threshold)
        objprop_t = objprop_t / objprop_t.sum()
        
        
        objprop_t = objprop_t.flatten().cpu()
        cum_objprop_t = np.cumsum(objprop_t)

        coordinates_x = []
        coordinates_y = []
        # map uniform samples to pixels
        for i, z in enumerate(np.random.uniform(size=num_samples)):
            diff = cum_objprop_t - z

            mask = np.ma.less_equal(diff, 0)
            masked_diff = np.ma.masked_array(diff, mask)

            # position in 1D-array where x is closest but greater than array value
            min_pos = masked_diff.argmin()

            # retrieve pixel coordinates
            x = min_pos % 256
            y = min_pos // 256

            coordinates_x.append(x)
            coordinates_y.append(y)
        

        xmin = torch.amin(contours[:, :, 1], dim=-1) 
        xmax = torch.amax(contours[:, :, 1], dim=-1)
        ymin = torch.amin(contours[:, :, 0], dim=-1)
        ymax = torch.amax(contours[:, :, 0], dim=-1)

        bboxes = torch.stack([xmin, ymin, xmax, ymax], -1)
        bboxes0 = torch.stack([xmin, ymin, xmax, ymax], -1)
        
        scores = objprop.reshape(256*256)

        bboxes = bboxes[scores>threshold]
        scores2 = scores[scores>threshold]

        selectedIds = batched_nms(bboxes, scores2, torch.ones(len(scores2)), iou_threshold=nms_threshold)

        bboxes = bboxes.detach().cpu().numpy()
        selectedIds = selectedIds.detach().cpu().numpy()

        rects = bboxes[selectedIds]
        
        
        scores3 = objprob.reshape(256*256)

        bboxes = bboxes0[scores3>threshold]
        scores4 = scores[scores3>threshold]

        selectedIds2 = batched_nms(bboxes, scores4, torch.ones(len(scores4)), iou_threshold=nms_threshold)

        bboxes = bboxes.detach().cpu().numpy()
        selectedIds2 = selectedIds2.detach().cpu().numpy()

        rects2 = bboxes[selectedIds2]
        
        

        # polygon IOU

#         scores = objectProbas[0].flatten(1).cpu().detach().numpy()
#         sortedInstances = np.argsort(scores, axis=1).copy()

#         til = [np.searchsorted(scores[0, sortedInstances[0]], threshold)]

#         contours2 = [contours[til[0]:]]
# #         print(til)
#         print(len(contours2[0]))
#         postProcessed = nonMaximumSuppresion(scores[0, sortedInstances[0, til[0]:]],
#                                             contours2[0],
#                                             threshold,
#                                             nms_threshold)

#         contoursSelected, selectedIds = postProcessed
#         selectedIds = np.array(selectedIds)
        
        
        # end of the non maximum suppression
        
        
        
        image = np.ones((256,256,3), np.uint8)*255
        image2 = np.uint8(denormalize(img[0].detach().cpu()).numpy().transpose(1, 2, 0)*255)
        
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
   
    
    #     image2 = clahe.apply(image2)
        img_yuv = cv2.cvtColor(image2, cv2.COLOR_RGB2YUV)

        # equalize the histogram of the Y channel
        img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])

        # convert the YUV image back to RGB format
        image2 = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
        
    #     image2 = cv2.equalizeHist(image2)
        ctrs = contours[scores>threshold][selectedIds].detach().cpu().numpy()
        ctrs2 = contours[scores3>threshold][selectedIds2].detach().cpu().numpy()

        fig, (ax2, ax3, ax4, ax5, ax6) = plt.subplots(
                    1, 5, figsize=(21, 13))
        

        
        ax4.set_title("Object Overlap Probaiblities")
        ax4.imshow(overlap.cpu(), cmap='gnuplot2')
        
        ax5.set_title("Object proposal ")
        ax5.imshow(objprop.cpu(), cmap='gnuplot2')
        

        
        ax2.set_title("Object Probabilities")
        # print(objectProbas.min(), objectProbas.max(), objectProbas.ptp())
        ax2.imshow(objprob.cpu(), cmap='gnuplot2')
        # plt.subplot(1,4,4)
        ax3.set_title("Object Instances")
        # instances, colors = getInstancesImageFromContours(objectContours)
        #ax4.imshow(denormalize(img[0]).cpu().numpy().transpose(1, 2, 0))
        ax3.imshow(image2)
        ax3.scatter(coordinates_x, coordinates_y, c=np.random.rand(len(coordinates_x),3), s=10)
        # ax4.imshow(instances, alpha=0.3)
        colors = [ list(map(lambda x: x/255, getRandomColor())) for j in range(len(contours))]

        for j, ct in enumerate(ctrs):
            ax3.fill(ctrs[j][:, 0], ctrs[j][:, 1], color=colors[j], alpha=0.1)
            ax3.plot(ctrs[j][:, 0], ctrs[j][:, 1], color=colors[j], linewidth=2, linestyle='dashed')
            
            
        ax6.imshow(image2)
        # ax4.imshow(instances, alpha=0.3)
#         colors = [ list(map(lambda x: x/255, getRandomColor())) for j in range(len(contours))]
        ax6.scatter(coordinates_x, coordinates_y, c=np.random.rand(len(coordinates_x),3), s=10)
        for j, ct in enumerate(ctrs):
            ax6.fill(ctrs2[j][:, 0], ctrs2[j][:, 1], color=colors[j], alpha=0.1)
            ax6.plot(ctrs2[j][:, 0], ctrs2[j][:, 1], color=colors[j], linewidth=2, linestyle='dashed')

        fig.tight_layout()
        plt.show()
            
